This is for importing all the important libraries required for the script.

In [63]:
import cv2
import numpy as np
import os

import databases of the wildfires

In [64]:
def load_images_and_labels(image_folder, label_folder):
    images = []
    labels = []
    filenames = sorted(os.listdir(image_folder))  # Ensure order
    
    first_image = cv2.imread(os.path.join(image_folder, filenames[0]))  # Read first image
    if first_image is None:
        raise ValueError("Error: Could not read first image.")
    
    target_size = (first_image.shape[1], first_image.shape[0])  # (width, height)

    for filename in filenames:
        img = cv2.imread(os.path.join(image_folder, filename))
        if img is not None:
            img = cv2.resize(img, target_size)  # Resize image to match first image size
            images.append(img)
        
        label_path = os.path.join(label_folder, filename.replace('.jpg', '.txt'))
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label_content = f.read().strip()
                labels.append(int(label_content) if label_content.isdigit() else 0)  # Ensure valid integer
        else:
            labels.append(0)  # Default to no fire if label is missing
    
    return images, labels

if the input is a video, extract frames from it

In [65]:

def extract_frames(video_path, temp_frame_folder):
    """ Extract frames from a video and save them as images. """
    if os.path.exists(temp_frame_folder):
        shutil.rmtree(temp_frame_folder)  # Clean up existing folder
    os.makedirs(temp_frame_folder)

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"⚠ Error: Cannot open video file {video_path}")

    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break  # Stop when video ends

        frame_filename = os.path.join(temp_frame_folder, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(frame_filename, frame)  # Save frame
        frame_count += 1

    cap.release()
    print(f"✅ Extracted {frame_count} frames from {video_path}.")

    return temp_frame_folder  # Return the folder containing extracted frames

process the images

In [ ]:
def process_images(image_folder, label_folder, output_filename, fps=24):
    # """ Process images for fire detection and generate a video output. """
    filenames = sorted(os.listdir(image_folder))  
    prev_lab = None
    computed_labels = []

    # Determine if labels exist
    actual_labels_available = os.path.exists(label_folder) and len(os.listdir(label_folder)) > 0  

    # Read first image to determine size
    first_image = cv2.imread(os.path.join(image_folder, filenames[0]))
    if first_image is None:
        raise ValueError("Error: Could not read the first image.")

    height, width = first_image.shape[:2]  
    # print(f"✅ Target Video Size: {width}x{height}")

    # Initialize VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  
    out = cv2.VideoWriter(output_filename, fourcc, fps, (width, height))

    if not out.isOpened():
        raise ValueError("⚠ Error: VideoWriter failed to initialize.")

    frame_count = 0

    for filename in filenames:
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Skipping unreadable image: {filename}")
            continue  

        img = cv2.resize(img, (width, height))  
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

        actual_label = None  
        if actual_labels_available:
            label_path = os.path.join(label_folder, filename.replace('.jpg', '.txt'))
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    label_content = f.read().strip()
                    actual_label = int(label_content) if label_content.isdigit() else 0

        if prev_lab is not None:
            diff = cv2.absdiff(lab, prev_lab)
            gray_diff = cv2.cvtColor(diff, cv2.COLOR_LAB2BGR)
            gray_diff = cv2.cvtColor(gray_diff, cv2.COLOR_BGR2GRAY)
            
            _, mask = cv2.threshold(gray_diff, 40, 255, cv2.THRESH_BINARY)  
            mask_sum = np.sum(mask)  
            
            computed_label = 1 if mask_sum > 5000 else 0  
            computed_labels.append((filename, computed_label, actual_label))

            print(f"Frame: {filename} | Fire Detected: {computed_label} | Mask Sum: {mask_sum}")

            # 🔥 Improved Fire Overlay
            fire_mask = np.zeros_like(img, dtype=np.uint8)  
            fire_mask[:, :, 2] = mask  

            blended = cv2.addWeighted(img, 0.7, fire_mask, 0.3, 0)  

            # Draw bounding boxes around detected fire
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            for contour in contours:
                if cv2.contourArea(contour) > 500:  
                    x, y, w, h = cv2.boundingRect(contour)
                    cv2.rectangle(blended, (x, y), (x + w, y + h), (0, 255, 255), 2)  

            # print(f"Writing Frame: {frame_count+1} | Shape: {img.shape} | Type: {img.dtype}")

            out.write(blended)  
            frame_count += 1  

        prev_lab = lab  

    out.release()

    if frame_count == 0:
        print("⚠ Warning: No frames were written to the video!")

    if actual_labels_available:
        valid_comparisons = [x for x in computed_labels if x[2] is not None]  
        correct = sum(1 for _, c, a in valid_comparisons if c == a)
        accuracy = correct / len(valid_comparisons) * 100 if valid_comparisons else 0
        print(f'✅ Accuracy: {accuracy:.2f}%')
    # else:
    #     print("⚠ No labels found. Skipping accuracy evaluation.")

main body

In [67]:
def main():
    dataset_path = "databases/FLAME_2/RGB Video 1.mp4"
    
    # Detect whether the input is a video or a folder
    if dataset_path.endswith((".mp4", ".avi", ".mov")):
        image_folder = extract_frames(dataset_path, "temp_frames")  
    else:
        image_folder = dataset_path  

    label_folder = os.path.join(dataset_path, "labels")

    process_images(image_folder, label_folder, "fire_detection.mp4")

if __name__ == "__main__":
    main()

🎥 Input detected as video. Extracting frames...
✅ Extracted 8731 frames from databases/FLAME_2/RGB Video 1.mp4.
✅ Target Video Size: 3840x2160
Frame: frame_0001.jpg | Fire Detected: 1 | Mask Sum: 2115072000
Writing Frame: 1 | Shape: (2160, 3840, 3) | Type: uint8
Frame: frame_0002.jpg | Fire Detected: 1 | Mask Sum: 2115072000
Writing Frame: 2 | Shape: (2160, 3840, 3) | Type: uint8
Frame: frame_0003.jpg | Fire Detected: 1 | Mask Sum: 2115072000
Writing Frame: 3 | Shape: (2160, 3840, 3) | Type: uint8
Frame: frame_0004.jpg | Fire Detected: 1 | Mask Sum: 2115072000
Writing Frame: 4 | Shape: (2160, 3840, 3) | Type: uint8
Frame: frame_0005.jpg | Fire Detected: 1 | Mask Sum: 2115072000
Writing Frame: 5 | Shape: (2160, 3840, 3) | Type: uint8
Frame: frame_0006.jpg | Fire Detected: 1 | Mask Sum: 2115072000
Writing Frame: 6 | Shape: (2160, 3840, 3) | Type: uint8
Frame: frame_0007.jpg | Fire Detected: 1 | Mask Sum: 2115072000
Writing Frame: 7 | Shape: (2160, 3840, 3) | Type: uint8
Frame: frame_0008

need to add frame parameter into the process_frames to be the same as input video, otherwise 24 fps 